### Item-base Collaborative Filtering - Retrieval
This notebook demonstrates how to retrieve the recommendations for a user using the Item-based Collaborative Filtering model in production. 

#### Pre-requisites
1. Have the index trained and saved in the `yelp_UserCF.db` file.

In [1]:
import sqlite3
import pickle

In [2]:
# Connect to the SQLite database
db_path = './yelp_UserCF.db'
conn = sqlite3.connect(db_path)

In [3]:
# Function to retrieve user-business mappings from the database
def retrieve_user_business_mapping(conn):
    cursor = conn.cursor()

    # Fetch user mappings
    cursor.execute('''SELECT user_id, user_idx FROM user_mapping''')
    user_mapping = {row[0]: row[1] for row in cursor.fetchall()}

    # Fetch business mappings
    cursor.execute('''SELECT business_id, business_idx FROM business_mapping''')
    business_mapping = {row[0]: row[1] for row in cursor.fetchall()}

    return user_mapping, business_mapping

In [4]:
# Retrieve the user-business mappings
user_mapping, business_mapping = retrieve_user_business_mapping(conn)

In [5]:
# Function to get businesses a user interacted with
def get_user_businesses(user_id, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT business_id, stars_review FROM user_item_index WHERE user_id = ?''', (user_id,))
    return cursor.fetchall()

In [6]:
# Function to get top-k similar users for a given user
def get_top_k_similar_users(user_id, k, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT similarity_vector FROM user_user_similarity WHERE user_id = ?''', (user_id,))
    result = cursor.fetchone()

    if result is None:
        return []

    similarity_vector = pickle.loads(result[0])
    indices, data = similarity_vector

    # Get top-k similar users
    top_k = sorted(zip(indices, data), key=lambda x: -x[1])[:k]

    # Map indices to user ids
    similar_users = [(list(user_mapping.keys())[idx], score) for idx, score in top_k]

    return similar_users

In [8]:
# Function to predict user interests based on similar users
def predict_user_interests(user_id, k=10, conn=conn):
    # Get top-k similar users
    similar_users = get_top_k_similar_users(user_id, k, conn)

    recommended_businesses = {}
    
    # For each similar user, get their business interactions
    for similar_user_id, _ in similar_users:
        user_businesses = get_user_businesses(similar_user_id, conn)

        for business_id, score in user_businesses:
            if business_id in recommended_businesses:
                recommended_businesses[business_id] += score
            else:
                recommended_businesses[business_id] = score

    # Sort recommendations by score
    recommended_businesses = sorted(recommended_businesses.items(), key=lambda x: -x[1])

    return recommended_businesses[:k]

In [9]:
# Get the top 10 users in the database
user_ids = list(user_mapping.keys())[:10]

for user_id in user_ids:
    recommendations = predict_user_interests(user_id, k=10)
    print(f"Recommendations for user {user_id}: {recommendations}")

# Close the database connection when done
conn.close()

Recommendations for user QEvaori3gwNeyPIQ4THl0Q: [('zSqTndm5Alq9dW1PyzbdTA', 40.0), ('UXVjbVfUUjlrZC0IXXiBMw', 5.0), ('xvI-E1zQAwjZPMv9PNY7xg', 5.0), ('_ab50qdWOk0DdB6XOrBitw', 5.0), ('_C7QiQQc47AOEv4PE3Kong', 5.0)]
Recommendations for user xaCoba8K8JbPlIuXvftaYw: [('vj6AetpADpHOYtMRZsXX3g', 41.0)]
Recommendations for user SCDydHUmVrwJOGRKXvi1iQ: [('UQMUtySn2q9cSObiw5UVsw', 22.0), ('f8MSbvd5JotxsFsS6fbl-Q', 12.0), ('0WkPwMoI1HxSP8jicwQOOQ', 5.0), ('5uR3C_gvSpC1BXFlqdHRLA', 5.0), ('9gPLN_mLR-kqwmcMe-QkOA', 5.0), ('LyV2VIBVyF2EUd3GEiD17A', 5.0), ('i76ERS3jM111T3VMbi2yrQ', 5.0), ('my8iGnzOiOO3__WCuRRlbQ', 5.0), ('sb8-TzsXOV7IsErbpHZo3g', 5.0), ('DI4OK-UVHZqCwpCE8jX2ow', 4.0)]
Recommendations for user 7zq1qhACzE8QGnVAfTbH_w: [('P-TC0VSnUsct3NhYrj5xOA', 32.0), ('9b-LZWsj75mmfMYuChXDkg', 13.0), ('YyFcefa9hIgVTmogyFKu9g', 5.0), ('knA3b_WnHUJAyrIzzL9Lrg', 5.0), ('EurtMI6FITq7KHmmO94pnA', 4.0), ('FqxfsOhn13BKlXm94f1bjA', 4.0), ('INQCr5GV3YKplwG8-EMK-A', 4.0), ('MLMJOHXAlcCrhMj8M4FBHg', 4.0), ('